In [1]:
import sagemaker
import os

In [20]:
import pathlib as pl

In [2]:
import datetime

In [3]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.debugger import TensorBoardOutputConfig

In [4]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
upload_data = False
if upload_data:
    data_input_path = sess.upload_data('data/severstal-binary')
else:
    data_input_path = "s3://sagemaker-us-east-2-475496805360/data"
print(data_input_path)

In [ ]:
training_input_path = data_input_path + "/train"
validation_input_path = data_input_path + "/validate"

In [ ]:
time_now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_logs_dir = f"s3://{sess.default_bucket()}/tensorboard_logs" # /fit/{time_now}" 

In [ ]:
tensorboard_logs_dir

In [28]:
file_list = [
    pl.Path(f)
    for f in sess.list_s3_files(sess.default_bucket(), "tensorboard_logs/fit/")
    if not f.endswith("/")
]

In [33]:
for f in file_list:
    print(f)
    sess.download_data(str(f.parent), sess.default_bucket(), str(f))

tensorboard_logs/fit/train/events.out.tfevents.1606126027.0eb86bb4c8f5.20.6033.v2
tensorboard_logs/fit/train/events.out.tfevents.1606126044.0eb86bb4c8f5.profile-empty
tensorboard_logs/fit/train/plugins/profile/2020_11_23_10_07_23/0eb86bb4c8f5.input_pipeline.pb
tensorboard_logs/fit/train/plugins/profile/2020_11_23_10_07_23/0eb86bb4c8f5.kernel_stats.pb
tensorboard_logs/fit/train/plugins/profile/2020_11_23_10_07_23/0eb86bb4c8f5.memory_profile.json.gz
tensorboard_logs/fit/train/plugins/profile/2020_11_23_10_07_23/0eb86bb4c8f5.overview_page.pb
tensorboard_logs/fit/train/plugins/profile/2020_11_23_10_07_23/0eb86bb4c8f5.tensorflow_stats.pb
tensorboard_logs/fit/train/plugins/profile/2020_11_23_10_07_23/0eb86bb4c8f5.trace.json.gz
tensorboard_logs/fit/train/plugins/profile/2020_11_23_10_07_23/0eb86bb4c8f5.xplane.pb
tensorboard_logs/fit/validation/events.out.tfevents.1606126036.0eb86bb4c8f5.20.14427.v2


In [ ]:
tensorboard_output_config = TensorBoardOutputConfig(s3_output_path=tensorboard_logs_dir)

In [ ]:
tf_estimator = TensorFlow(
    entry_point='fine_tune_eff_net.py', 
    role=role,
    instance_count=1, 
    instance_type='local',  # local 'ml.g4dn.xlarge'
    framework_version='2.3', 
    py_version='py37',
    script_mode=True,
    hyperparameters={'epochs': 2},
    tensorboard_output_config=tensorboard_output_config
)

In [ ]:
tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path, 'log-dir': tensorboard_logs_dir})